# Notebook para pre processar o arquivo parquet gerado pelo agregrator

## Importação bibliotecas

In [1]:
!pip install pyarrow
!pip install fastparquet
!pip install pandas
!pip install numpy
!pip install seaborn
!pip install matplotlib
!pip install scikit-learn
!pip install scipy.stats
!pip install dask dask[dataframe]
!pip install tslearn
!pip install tensorflow

ERROR: Could not find a version that satisfies the requirement scipy.stats (from versions: none)
ERROR: No matching distribution found for scipy.stats


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import scipy.stats as stats
import os
import dask.dataframe as dd

## Carregamento do arquivo parquet

In [3]:
def load_parquet_file(file_path):
    if os.path.exists(file_path) and file_path.endswith('.parquet'):
        df = pd.read_parquet(file_path)
        print(f"O arquivo tem {len(df)} linhas.")
        return df
    else:
        print("Caminho do arquivo inválido ou não é um arquivo Parquet.")
        return None

# Uso da função
file_path = "new_data_voo.parquet"
preprocessing_df = load_parquet_file(file_path)

O arquivo tem 1711622 linhas.


In [4]:
preprocessing_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1711622 entries, 0 to 11758
Data columns (total 50 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   recording_time             int64         
 1   data_voo                   datetime64[ns]
 2   phaseOfFlight-1            float64       
 3   message0418DAA-1           float64       
 4   message0422DAA-1           float64       
 5   amscHprsovDrivF-1a         float64       
 6   amscHprsovDrivF-1b         float64       
 7   amscHprsovDrivF-2b         float64       
 8   amscPrsovDrivF-1a          float64       
 9   amscPrsovDrivF-1b          float64       
 10  amscPrsovDrivF-2b          float64       
 11  basBleedLowPressF-1a       float64       
 12  basBleedLowPressF-2b       float64       
 13  basBleedLowTempF-1a        float64       
 14  basBleedLowTempF-2b        float64       
 15  basBleedOverPressF-1a      float64       
 16  basBleedOverPressF-2b      float64       
 

In [5]:
preprocessing_df.describe()

,recording_time,data_voo,phaseOfFlight-1,message0418DAA-1,message0422DAA-1,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,amscPrsovDrivF-1b,...,bleedOnStatus-2b,bleedOverpressCas-2a,bleedOverpressCas-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a
count,1.711622e+06,83885,577243.000000,83885.0,83885.000000,1006690.0,1006690.0,1.006690e+06,1006690.0,1006690.0,...,577243.000000,1006690.0,1006690.0,1.006690e+06,1.006690e+06,1.006690e+06,1.006690e+06,577243.000000,577243.000000,1.006690e+06
mean,3.790776e+06,2022-12-29 03:03:31.532455424,0.812270,0.0,0.059784,0.0,0.0,1.192025e-05,0.0,0.0,...,0.827405,0.0,0.0,5.797034e-01,5.800514e-01,1.237487e-01,1.630250e-03,0.130635,0.161958,4.484631e+01
min,0.000000e+00,2022-12-02 00:00:00,0.000000,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.0,...,0.000000,0.0,0.0,-2.500000e-01,-2.500000e-01,-1.280000e+02,-1.280000e+02,0.000000,0.000000,2.500000e-01
25%,1.561300e+06,2022-12-17 00:00:00,0.000000,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.0,...,1.000000,0.0,0.0,4.375000e-01,4.375000e-01,3.750000e-01,3.750000e-01,0.000000,0.000000,7.500000e-01
50%,3.284350e+06,2022-12-30 00:00:00,1.000000,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.0,...,1.000000,0.0,0.0,6.250000e-01,6.250000e-01,5.625000e-01,5.625000e-01,0.000000,0.000000,8.750000e-01
75%,5.650550e+06,2023-01-15 00:00:00,2.000000,0.0,0.000000,0.0,0.0,0.000000e+00,0.0,0.0,...,1.000000,0.0,0.0,7.500000e-01,7.500000e-01,6.562500e-01,6.562500e-01,0.000000,0.000000,1.467500e+02
max,1.095300e+07,2023-01-18 00:00:00,2.000000,0.0,7.000000,0.0,0.0,1.000000e+00,0.0,0.0,...,1.000000,0.0,0.0,3.875000e+00,4.187500e+00,4.687500e+00,4.687500e+00,1.000000,1.000000,3.403750e+02
std,2.703526e+06,NaN,0.820696,0.0,0.642661,0.0,0.0,3.452553e-03,0.0,0.0,...,0.377897,0.0,0.0,3.370323e-01,3.369332e-01,7.092480e+00,8.120509e+00,0.337001,0.368412,7.243419e+01


## Remoção de valores nulos das colunas dos sensores

In [6]:
columns_to_fill = ['message0418DAA-1','message0422DAA-1','amscHprsovDrivF-1a', 'amscHprsovDrivF-1b',
                   'amscHprsovDrivF-2b', 'amscPrsovDrivF-1a',
                   'amscPrsovDrivF-1b', 'amscPrsovDrivF-2b',
                   'basBleedLowPressF-1a', 'basBleedLowPressF-2b',
                   'basBleedLowTempF-1a', 'basBleedLowTempF-2b',
                   'basBleedOverPressF-1a', 'basBleedOverPressF-2b',
                   'basBleedOverTempF-1a', 'basBleedOverTempF-2b',
                   'bleedFavTmCmd-1a', 'bleedFavTmCmd-1b',
                   'bleedFavTmCmd-2a', 'bleedFavTmCmd-2b', 'bleedFavTmFbk-1a',
                   'bleedFavTmFbk-1b', 'bleedFavTmFbk-2b', 'bleedHprsovCmdStatus-1a',
                   'bleedHprsovCmdStatus-1b', 'bleedHprsovCmdStatus-2a',
                   'bleedHprsovCmdStatus-2b', 'bleedHprsovOpPosStatus-1a',
                   'bleedHprsovOpPosStatus-1b', 'bleedHprsovOpPosStatus-2a',
                   'bleedHprsovOpPosStatus-2b', 'bleedMonPress-1a',
                   'bleedMonPress-1b', 'bleedMonPress-2a', 'bleedMonPress-2b',
                   'bleedOnStatus-1a', 'bleedOnStatus-1b', 'bleedOnStatus-2b',
                   'bleedOverpressCas-2a', 'bleedOverpressCas-2b',
                   'bleedPrecoolDiffPress-1a', 'bleedPrecoolDiffPress-1b',
                   'bleedPrecoolDiffPress-2a', 'bleedPrecoolDiffPress-2b',
                   'bleedPrsovClPosStatus-1a', 'bleedPrsovClPosStatus-2a',
                   'bleedPrsovFbk-1a']

# Criando um dicionário que mapeia cada coluna ao valor 0
values_dict = {col: 0 for col in columns_to_fill}

# Usando o método fillna() para substituir os NaNs
preprocessing_df.fillna(value=values_dict, inplace=True)

In [7]:
# Verificando a quantidade de NaNs em cada coluna especificada
nan_counts = preprocessing_df[columns_to_fill].isna().sum()

# Filtrando as colunas que têm qualquer NaN
columns_with_nans = nan_counts[nan_counts > 0]

if columns_with_nans.empty:
    print("Nenhuma das colunas especificadas tem valores nulos.")
else:
    print("Quantidade de valores nulos por coluna:")
    print(columns_with_nans)

Nenhuma das colunas especificadas tem valores nulos.


## Criação de uma coluna derivada da coluna record_time identificando os períodos de milisegundos por hora

In [12]:
# Quantos milissegundos existem em 1 hora
ms_per_hour = 3600 * 1000

# Número de registros por hora
records_per_hour = ms_per_hour / 50

# Criar um novo DataFrame baseado no preprocessing_df
horas_voo_df = preprocessing_df.copy()

# Adicionar a coluna 'horas_voo' ao novo DataFrame baseado na coluna 'recording_time'
horas_voo_df['horas_voo'] = (horas_voo.index // records_per_hour) + 1

# Reordenar as colunas para que 'horas_voo' fique ao lado de 'recording_time'
cols = new_df.columns.tolist()
idx = cols.index('recording_time')
cols.insert(idx+1, cols.pop(cols.index('horas_voo')))
horas_voo_df = horas_voo_df[cols]

In [13]:
horas_voo_df.head()

,recording_time,horas_voo,data_voo,phaseOfFlight-1,message0418DAA-1,message0422DAA-1,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,...,bleedOnStatus-2b,bleedOverpressCas-2a,bleedOverpressCas-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
0,0,1.0,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000,1.0,1.0,0.00
1,50,1.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.03125,0.03125,0.03125,0.03125,0.0,0.0,1.75
2,100,1.0,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000,1.0,1.0,0.00
3,150,1.0,2022-12-06,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000,0.0,0.0,0.00
4,200,1.0,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000,1.0,1.0,0.00


## Exclusão de colunas de sensores que não possuem variação nos valores dos dados

In [14]:
# Lista das colunas para verificação
columns_to_check = [
    'amscHprsovDrivF-1a', 'amscHprsovDrivF-1b', 'amscHprsovDrivF-2b', 'amscPrsovDrivF-1a',
    'amscPrsovDrivF-1b', 'amscPrsovDrivF-2b', 'basBleedLowPressF-1a', 'basBleedLowPressF-2b',
    'basBleedLowTempF-1a', 'basBleedLowTempF-2b', 'basBleedOverPressF-1a', 'basBleedOverPressF-2b',
    'basBleedOverTempF-1a', 'basBleedOverTempF-2b', 'bleedFavTmCmd-1a', 'bleedFavTmCmd-1b',
    'bleedFavTmCmd-2a', 'bleedFavTmCmd-2b', 'bleedFavTmFbk-1a', 'bleedFavTmFbk-1b', 'bleedFavTmFbk-2b',
    'bleedHprsovCmdStatus-1a', 'bleedHprsovCmdStatus-1b', 'bleedHprsovCmdStatus-2a', 'bleedHprsovCmdStatus-2b',
    'bleedHprsovOpPosStatus-1a', 'bleedHprsovOpPosStatus-1b', 'bleedHprsovOpPosStatus-2a', 'bleedHprsovOpPosStatus-2b',
    'bleedMonPress-1a', 'bleedMonPress-1b', 'bleedMonPress-2a', 'bleedMonPress-2b', 'bleedOnStatus-1a',
    'bleedOnStatus-1b', 'bleedOnStatus-2b', 'bleedOverpressCas-2a', 'bleedOverpressCas-2b',
    'bleedPrecoolDiffPress-1a', 'bleedPrecoolDiffPress-1b', 'bleedPrecoolDiffPress-2a', 'bleedPrecoolDiffPress-2b',
    'bleedPrsovClPosStatus-1a', 'bleedPrsovClPosStatus-2a', 'bleedPrsovFbk-1a'
]

# Identificar colunas sem variação
non_varying_columns = [col for col in columns_to_check if horas_voo_df[col].nunique() == 1]

# Remover colunas sem variação do DataFrame
variação_df = horas_voo_df.drop(columns=non_varying_columns)

In [15]:
# Lista das colunas que foram removidas
print("Colunas removidas:")
for col in non_varying_columns:
    print(col)

# Lista das colunas que foram mantidas
remaining_columns = [col for col in columns_to_check if col not in non_varying_columns]

print("\nColunas mantidas:")
for col in remaining_columns:
    print(col)

Colunas removidas:
amscHprsovDrivF-1a
amscHprsovDrivF-1b
amscPrsovDrivF-1a
amscPrsovDrivF-1b
amscPrsovDrivF-2b
basBleedLowPressF-1a
basBleedLowPressF-2b
basBleedLowTempF-1a
basBleedLowTempF-2b
basBleedOverPressF-1a
basBleedOverPressF-2b
basBleedOverTempF-1a
bleedOverpressCas-2a
bleedOverpressCas-2b

Colunas mantidas:
amscHprsovDrivF-2b
basBleedOverTempF-2b
bleedFavTmCmd-1a
bleedFavTmCmd-1b
bleedFavTmCmd-2a
bleedFavTmCmd-2b
bleedFavTmFbk-1a
bleedFavTmFbk-1b
bleedFavTmFbk-2b
bleedHprsovCmdStatus-1a
bleedHprsovCmdStatus-1b
bleedHprsovCmdStatus-2a
bleedHprsovCmdStatus-2b
bleedHprsovOpPosStatus-1a
bleedHprsovOpPosStatus-1b
bleedHprsovOpPosStatus-2a
bleedHprsovOpPosStatus-2b
bleedMonPress-1a
bleedMonPress-1b
bleedMonPress-2a
bleedMonPress-2b
bleedOnStatus-1a
bleedOnStatus-1b
bleedOnStatus-2b
bleedPrecoolDiffPress-1a
bleedPrecoolDiffPress-1b
bleedPrecoolDiffPress-2a
bleedPrecoolDiffPress-2b
bleedPrsovClPosStatus-1a
bleedPrsovClPosStatus-2a
bleedPrsovFbk-1a


In [16]:
variação_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1711622 entries, 0 to 11758
Data columns (total 37 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   recording_time             int64         
 1   horas_voo                  float64       
 2   data_voo                   datetime64[ns]
 3   phaseOfFlight-1            float64       
 4   message0418DAA-1           float64       
 5   message0422DAA-1           float64       
 6   amscHprsovDrivF-2b         float64       
 7   basBleedOverTempF-2b       float64       
 8   bleedFavTmCmd-1a           float64       
 9   bleedFavTmCmd-1b           float64       
 10  bleedFavTmCmd-2a           float64       
 11  bleedFavTmCmd-2b           float64       
 12  bleedFavTmFbk-1a           float64       
 13  bleedFavTmFbk-1b           float64       
 14  bleedFavTmFbk-2b           float64       
 15  bleedHprsovCmdStatus-1a    float64       
 16  bleedHprsovCmdStatus-1b    float64       
 